In [ ]:
from transformers import MT5ForConditionalGeneration as MT5Model
from transformers import AdamW
from transformers import MT5Tokenizer
import pickle

In [ ]:
model = MT5Model.from_pretrained("google/mt5-small")
model.train()
tokenizer = MT5Tokenizer.from_pretrained('google/mt5-small')
optimizer = AdamW(model.parameters(), lr=0.00001)
print("Number of parameters: ", model.num_parameters())

In [ ]:
with open("/kaggle/input/tolokadialogues/prepared_dialogueCR.pickle", 'rb') as file:
    dialogues = pickle.load(file)

In [ ]:
## even stands for even number of repliques in one dialog, so each U1 replique has an answer
dialogues_even = list(map(lambda x: x[:-1] if len(x) % 2 == 1 else x, dialogues))
dialogues_even_flatten = [replique for dialogue in dialogues for replique in dialogue]
repliquesU1 = list(filter(lambda x: x.startswith("USER1:"), dialogues_even_flatten))
repliquesU2 = list(filter(lambda x: x.startswith("USER2:"), dialogues_even_flatten))

In [ ]:
data = tokenizer.prepare_seq2seq_batch(src_texts=repliquesU1[:100], tgt_texts=repliquesU2[:100], return_tensors="pt", padding=True)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir = "garbage/",
    num_train_epochs=1,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=data
)

In [ ]:
trainer.train()

In [ ]:
model.save_pretrained('models/mt5small/')